In [1]:
# import library
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import mne
from mne import Epochs, pick_types, find_events, pick_types, set_eeg_reference
from mne.channels import read_layout
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
from mne import viz

In [3]:
# read data
myPath = r'D:\Thesis\Capstone-BCI-13\dataset\chb15\chb15_06.edf'

# basic info
tmin, tmax = -0.1, 1.0
raw = read_raw_edf(myPath, preload=True,
                   stim_channel=None)  # can use easy path
print("-----------------------------------------------------------------")
print(f'Data type: {type(raw)}\n\n{raw}')
print('Sampling rate:', raw.info['sfreq'], 'Hz')
print(f'Size of all channels  {raw.get_data().shape}')

# Choose channel f8-t8
f8_t8_channel_info = raw.pick_channels(['F8-T8'])
print(
    f'Size of F8-T8 channel {f8_t8_channel_info.get_data().shape} and the info are\n')
print(f8_t8_channel_info)


Extracting EDF parameters from D:\Thesis\Capstone-BCI-13\dataset\chb15\chb15_06.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\Admin\AppData\Local\Temp/ipykernel_32924/2633913067.py:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  raw = read_raw_edf(myPath, preload=True,
C:\Users\Admin\AppData\Local\Temp/ipykernel_32924/2633913067.py:6: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4, --5
  raw = read_raw_edf(myPath, preload=True,


Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
-----------------------------------------------------------------
Data type: <class 'mne.io.edf.edf.RawEDF'>

<RawEDF | chb15_06.edf, 38 x 921600 (3600.0 s), ~267.2 MB, data loaded>
Sampling rate: 256.0 Hz
Size of all channels  (38, 921600)
Size of F8-T8 channel (1, 921600) and the info are

<RawEDF | chb15_06.edf, 1 x 921600 (3600.0 s), ~7.0 MB, data loaded>


In [21]:
# Create X, y from f8_t8_channel_info
seizure_time = {"start": 272, "stop": 397}
state = {"preictal": 0, "ictal": 1, "interictal": 2}
X = f8_t8_channel_info.get_data()
X = X.transpose()
X = X[0:1000] 
y = np.zeros((1000,), dtype=int)
y[:seizure_time["start"]-1] = state["preictal"]
y[seizure_time["start"]:seizure_time["stop"]] = state["ictal"]
y[seizure_time["stop"]+1:] = state["interictal"]

print(f'Type of X is {type(X)}')
print(f'Size of X is {X.shape}')
print(y.shape)
print(X.size)

Type of X is <class 'numpy.ndarray'>
Size of X is (1000, 1)
(1000,)
1000


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
  
# training a linear SVM classifier
from sklearn.svm import SVC
svm_model_linear = SVC(kernel = 'linear', C = 1, max_iter = 10).fit(X_train, y_train)
svm_predictions = svm_model_linear.predict(X_test)
  
# model accuracy for X_test  
accuracy = svm_model_linear.score(X_test, y_test)
  
# creating a confusion matrix
cm = confusion_matrix(y_test, svm_predictions)

c:\Users\Admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


In [28]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, svm_predictions)
print('Confusion Matrix\n')
print(confusion)

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, svm_predictions)))

print('Micro Precision: {:.2f}'.format(precision_score(y_test, svm_predictions, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_test, svm_predictions, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, svm_predictions, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_test, svm_predictions, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_test, svm_predictions, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, svm_predictions, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_test, svm_predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test, svm_predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, svm_predictions, average='weighted')))

from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_test, svm_predictions, target_names=['Class 1', 'Class 2', 'Class 3']))


Confusion Matrix

[[28  0 18]
 [23  0  6]
 [73  0 52]]

Accuracy: 0.40

Micro Precision: 0.40
Micro Recall: 0.40
Micro F1-score: 0.40

Macro Precision: 0.30
Macro Recall: 0.34
Macro F1-score: 0.28

Weighted Precision: 0.48
Weighted Recall: 0.40
Weighted F1-score: 0.40

Classification Report

              precision    recall  f1-score   support

     Class 1       0.23      0.61      0.33        46
     Class 2       0.00      0.00      0.00        29
     Class 3       0.68      0.42      0.52       125

    accuracy                           0.40       200
   macro avg       0.30      0.34      0.28       200
weighted avg       0.48      0.40      0.40       200



c:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:124